In [1]:
#importing Libraries

import os
import numpy as np
import pandas as pd

from nltk.corpus import wordnet as wn
import nltk
from scipy.stats import spearmanr as spc
from scipy.stats import pearsonr as pec
from sematch.semantic.similarity import WordNetSimilarity
wns = WordNetSimilarity()

In [2]:
def ESASim(text1,text2,noConcepts):
    #You will get top noConcepts in concept1 and concept2

    ###! Discalimer Do not touch support code
    loc = "/home/sp/esaexpfolder/storage/"
    prog = "/home/sp/NetBeansProjects/ESA/dist/ESA.jar"
    indexDir = "/home/sp/NLP-PA1/descartes-0.2/data/wiki-index"

    f1outname = "concept1.csv"
    f1outname = loc + f1outname

    f2outname = "concept2.csv"
    f2outname = loc + f2outname
    #Cleaning saving location script
    clrData = "rm "+loc+"*"
    os.system(clrData)


    fileLoc1 = loc+"text1.txt"
    f1 = open(fileLoc1,"w",encoding="utf-8")
    f1.write(text1)
    f1.close()

    fileLoc2 = loc+"text2.txt"
    f2 = open(fileLoc2,"w",encoding="utf-8")
    f2.write(text2)
    f2.close()

    #For Concept form ESA
    f1script = "java -jar"+ " " + prog + " " + indexDir + " " + str(noConcepts) + " " + fileLoc1 + " " + f1outname  
    os.system(f1script)

    f2script = "java -jar"+ " " + prog + " " + indexDir + " " + str(noConcepts) + " " + fileLoc2 + " " + f2outname  
    os.system(f2script)

    concept1 = pd.read_csv(f1outname)
    concept2 = pd.read_csv(f2outname)

    conval1 = concept1.values
    conval2 = concept2.values

    set1 = set(conval1[:,0])
    set2 = set(conval2[:,0])

    A = set1.intersection(set2)
    B = set1.union(set2)
    Alen = len(A)
    Blen = len(B)
    
    i = 0
    dictSpace1 = {}
    for val in A:
        dictSpace1[val] = i
        i = i + 1

    ivec1 = np.zeros(len(A))
    ivec2 = np.zeros(len(A))

    for v in dictSpace1.keys():
        gIn = dictSpace1[v]
        ind1 = list(conval1[:,0]).index(v)
        ivec1[gIn] = conval1[ind1,1]

        ind2 = list(conval2[:,0]).index(v)
        ivec2[gIn] = conval2[ind2,1]

    i = 0
    dictSpace = {}
    for val in B:
        dictSpace[val] = i
        i = i + 1

    vect1 = np.zeros(len(B))
    vect2 = np.zeros(len(B))

    for x,y in zip(conval1[:,0],conval1[:,1]):
        ind = dictSpace[x]
        vect1[ind] = y

    for x,y in zip(conval2[:,0],conval2[:,1]):
        ind = dictSpace[x]
        vect2[ind] = y

    vect1 = np.array(vect1)
    vect2 = np.array(vect2)

    #s1 = np.sum(ivec1*ivec2)/(np.sqrt(np.sum(ivec1*ivec1)) * np.sqrt(np.sum(ivec2*ivec2)))
    #s1 = s1/np.sqrt(Blen-Alen+1)
    s2 = np.sum(vect1*vect2)/(np.sqrt(np.sum(vect1*vect1)) * np.sqrt(np.sum(vect2*vect2)))
    return s2

In [10]:
def pathsim(w1,w2):
    #Calculating Shortest Path between concepts
    distance = w1.shortest_path_distance(w2)
    
    #In case doesn't have a path
    if distance == None:
        return None
    #Path exists, Assuming longer the distance lesser the similarity
    return 1/(distance+1.0)

basOp = pathsim

# To calculate similarity between two english word based on word.
# Since lemma is not defined we will consider the similary which will be the best among path.
# Word1,Word2 : are the 2 words between which we have to find similarity
# SimOp : The similarity measure we wish to apply.
### SimOp : default : basOp(basic path based similarity nothing fancy), lch_similarity, wup_similarity 
def wordsimCalc(word1,word2,simOp=basOp):
    d1 = '' #defination of the current word1
    d2 = '' #defination of the current word2
    fd1 = '' #defination of the wordform that found max sim
    fd2 = '' #defination of the wordform that found max sim
    wsl1 = wn.synsets(word1)
    wsl2 = wn.synsets(word2)
    maxSim = 0
    for i in wsl1:
        for j in wsl2:
            d1 = i.definition()
            d2 = j.definition()
            # Words should belong to same to tree structure.
            # Noun to Noun, Verb to Verb. There is no path between Noun to Verb
            # There is no IS-A link structure between Adjective, so they have to be dropen
            if (i.pos() == 'v' or i.pos()=='n') and (j.pos() == 'v' or j.pos()=='n') and (i.pos() == j.pos()) :
                #sim = ESASim(word1+" "+d1,word2+" "+d2,100)
                sim = simOp(i,j)
                #If path doesn't exist in the taxonomy.
                if sim == None:
                    continue
                #If sim is greater than previous existed 
                if sim > maxSim:
                    maxSim = sim
                    fd1 = d1
                    fd2 = d2
    
    return (maxSim,fd1,fd2)
listpp = [10,20,50,100,200,500]
indd = 0
corelp = np.zeros(len(listpp))
for pp in listpp:
    wordSimData = pd.read_csv('/home/sp/NLP_PA1/path_sim.csv')
    w1list = wordSimData.values[:,0]
    w2list = wordSimData.values[:,1]
    humanSim = wordSimData.values[:,2]

    calSim = np.zeros(len(w1list))
    ccc = 0
    i = 0
    for w1,w2 in zip(w1list,w2list):
        ccc,def1,def2 = wordsimCalc(w1,w2,simOp = wn.wup_similarity)
        text1 = w1 + " " + def1
        text2 = w2 + " " + def2
        calSim[i] = ESASim(text1,text2,pp)
        #print(i ,"-- ", w1," : ",def1 ," <> ", w2 ," : ",def2,"\n")
        print(i)
        i = i + 1

    cors,pvals = spc(calSim,(humanSim))
    corelp[indd] = cors
    indd = indd + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
cors,pvals = spc(calSim,(humanSim))
corp,pvalp = pec(calSim,(humanSim))

In [11]:
corelp


array([0.34224648, 0.42498808, 0.45235273, 0.45778123, 0.50823641,
       0.52521774])